In [1]:
import requests
import re
import json
import geopandas as gpd
import folium
import plotly.express as px
import panel as pn
import param
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from enum import Enum
from dash import Dash, html, Input, Output, State, dcc
from shapely.geometry import LineString, Point, shape
from shapely.ops import nearest_points

pn.extension('plotly')

# Example forecast call
# https://opendata.fmi.fi/download?producer=harmonie_scandinavia_surface&param=PrecipitationAmount&bbox=24,60,25,61&origintime=2023-04-15T06:00:00Z&starttime=2023-04-15T06:00:00Z&endtime=2023-04-18T00:00:00Z&format=grib2&projection=EPSG:4326&levels=0&timestep=60

In [25]:
def get_paginated_items(url, verbose=False):
    data = []
    while url is not None:
        resp = requests.get(url)
        resp.raise_for_status()
        resp = resp.json()

        data.extend(resp['results'])
        url = resp['next']
        if verbose:
            print(url)  
    return data

url = 'https://api.hel.fi/servicemap/v2/unit/?service=737'
units = get_paginated_items(url)
filtered_places = [u for u in units if re.search(r'/ Ulkokuntosali', u['name']['fi'])]

KeyboardInterrupt: 

In [ ]:
def get_paths(start, end, path_type):
    url = f"https://www.greenpaths.fi/paths/walk/{path_type.lower()}/{start.x},{start.y}/{end.x},{end.y}"
    resp = requests.get(url)
    data = resp.json()
    return data

def find_n_closest(orig, destinations, n=5):
    points = [(shape(d['location']), ind)for ind, d in enumerate(destinations)]
    points = [(Point(p[0].y,p[0].x), p[1]) for p in points]
    distances = [(orig.distance(p[0]),p[1]) for p in points]
    distances = sorted(distances, key=lambda x: x[0])
    
    # TODO: Actually return n closest instead of the closest
    return [points[d[1]] for d in distances[:n]]

# origin = Point(60.20772,24.96716)

# closest_gyms = find_n_closest(origin, filtered_places, 5)
# closest_gym_coord = closest_gyms[0][0]

# data = get_paths(origin, closest_gym_coord)

# print(shape(data['path_FC']['features'][0]['geometry']))

# for p in filtered_places:
#     print(p['name']['fi'])
#     # print(p['location'])
#     print(shape(p['location']))

In [ ]:
app = JupyterDash(prevent_initial_callbacks=True)
app.layout = html.Div([
    html.Div(dcc.Dropdown(['Fast', 'Green', 'Quiet', 'Clean'], 'Fast', id='path-pref')),
    html.Div([
    dl.Map([dl.TileLayer(), dl.LayerGroup(id="marker-layer"), dl.LayerGroup(id='path-layer')],
           center=[60.1699,24.9384],
           zoom=13,
           id="map", style={'width': '100%', 'height': '75vh', 'margin': "auto", "display": "block"}),
    html.Button(id='submit-button-state', n_clicks=0, children='Confirm position'),
    html.Div(id='hidden-div', style={'display':'none'})
    ])
])


@app.callback(Output("marker-layer", "children"),
             [Input("map", "click_lat_lng")])
def map_click(click_lat_lng):
    return [dl.Marker(id='start-marker', position=click_lat_lng, children=dl.Tooltip("({:.3f}, {:.3f})".format(*click_lat_lng)))]

@app.callback(Output('path-layer', 'children'),
              Input('submit-button-state', 'n_clicks'),
              State('start-marker', 'position'),
              State('path-pref', 'value')
              )
def button_click(n_clicks, coords, path_pref):
    print(f"Button clicked {n_clicks} times")
    origin = Point(coords)
    closest_gyms = find_n_closest(origin, filtered_places, 5)
    closest_gym_coord = closest_gyms[0][0]
    data = get_paths(origin, closest_gym_coord, path_pref)
    path = [[c[1], c[0]] for c in data['path_FC']['features'][0]['geometry']['coordinates']]
    print(path_pref)

    return [dl.Polyline(positions=path),dl.Marker(position=[closest_gym_coord.x, closest_gym_coord.y])]

# Button click
# 1. Get the origin
# 2. Find n closest points of interest (to make search more feasible)
# 3. Calculate the paths of desired type to each of the POIs
# 4. Get the best POI/path according to the desired criteria (fastest, quietest, cleanest, etc.)



if __name__ == '__main__':
    app.run_server(mode="inline")

Dash is running on http://127.0.0.1:8050/



Button clicked 1 times
Fast
Button clicked 2 times
Quiet


In [ ]:
# callback = lambda x,y: print(f"lat: {x}, long: {y}")
callback = lambda x: print(x)
base_map = px.line_mapbox(lat=[0], lon=[0],
                           mapbox_style='open-street-map', 
                           center={"lat": 60.1699, "lon": 24.9384},
                           zoom=12)
# base_map = folium.Map(location=[60.1699,24.9384], tiles='OpenStreetMap')
# base_map.add_child(folium.LatLngPopup())
# pane = pn.pane.plot.Folium(base_map, height=400)

# pane.jscallback(clicks="""
# print('hello')
# """)
pane = pn.pane.Plotly(base_map)

pane.param.watch(callback, ['click_data'])

pane.on_click(callback)
pane

AttributeError: 'Plotly' object has no attribute 'on_click'

Button clicked 1 times
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
KeyError: 'path_FC'

Button clicked 2 times
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
KeyError: 'path_FC'



In [ ]:
# Try out the green paths API
test_url = "https://www.greenpaths.fi/paths/walk/clean/60.20772,24.96716/60.2037,24.9653"
resp = requests.get(test_url)

In [ ]:
data = resp.json()
print(json.dumps(data['path_FC']['features'], indent=4))
# print(data['path_FC']['features'][0]['properties']['id'])

# print(data['path_FC']['features'][0]['geometry'])
# print(shape(data['path_FC']['features'][0]['geometry']))
# proposed_path = gpd.GeoSeries(LineString(data['path_FC']['features'][0]['geometry']['coordinates']))
# proposed_path
# proposed_path.crs = 4326
# proposed_path.explore()


[
    {
        "geometry": {
            "coordinates": [
                [
                    24.967048,
                    60.20781
                ],
                [
                    24.967081,
                    60.207826
                ],
                [
                    24.967081,
                    60.207826
                ],
                [
                    24.967415,
                    60.207779
                ],
                [
                    24.967415,
                    60.207779
                ],
                [
                    24.967592,
                    60.207762
                ],
                [
                    24.967592,
                    60.207762
                ],
                [
                    24.967631,
                    60.207753
                ],
                [
                    24.967631,
                    60.207753
                ],
                [
                    24.967722,
           

In [ ]:
# Load the list of serices
with open('all_services.json', 'r') as f:
    services = json.load(f)
services = services['data']

In [ ]:
for s in services:
    if s['id'] == 737:
        print(json.dumps(s, indent=4, ensure_ascii=False))

{
    "name": {
        "fi": "ulkokuntoiluvälineet",
        "sv": "redskap för utomhusmotion",
        "en": "outdoor fitness training equipment"
    },
    "id": 737,
    "period_enabled": false,
    "clarification_enabled": false,
    "keywords": {
        "fi": [
            "kuntoiluvälineet",
            "kuntonurkat",
            "kuntonurkka",
            "ulkosalit",
            "ulkoverka",
            "väliverkka"
        ]
    },
    "root_service_node": 1403,
    "unit_count": {
        "municipality": {
            "vantaa": 66,
            "kirkkonummi": 1,
            "kauniainen": 5,
            "espoo": 116,
            "helsinki": 176,
            "vihti": 3
        },
        "organization": {
            "espoon kaupunki": 105,
            "vantaan kaupunki": 66,
            "kauniaisten kaupunki": 5,
            "helsingin kaupunki": 190,
            "kirkkonummi": 1
        },
        "total": 367
    }
}


In [ ]:
url = 'https://api.hel.fi/servicemap/v2/unit/?service=737'
units = []

while url is not None:
    resp = requests.get(url)
    resp.raise_for_status()
    resp = resp.json()

    units.extend(resp['results'])
    url = resp['next']
    print(url)

filtered_places = [u for u in units if re.search(r'/ Ulkokuntosali', u['name']['fi'])]

In [ ]:
for p in filtered_places:
    print(p['name']['fi'])
    print(p['location'])


Arabianrannan liikuntapuisto / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.978802, 60.20493]}
Eira Open Air / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.938833, 60.153976]}
Puotilankenttä / Ulkokuntosali
{'type': 'Point', 'coordinates': [25.096191, 60.21545]}
Kurkimäen liikuntapuisto / Ulkokuntosali
{'type': 'Point', 'coordinates': [25.067743, 60.23415]}
Rautaranta Outdoor Gym Helsinki / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.963285, 60.18248]}
Pikkukosken uimaranta / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.982218, 60.22775]}
Aurinkolahden uimaranta / Ulkokuntosali (2/2)
{'type': 'Point', 'coordinates': [25.160913, 60.202694]}
Paloheinän ulkoilualue / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.916718, 60.254833]}
The Park Hietsu / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.906046, 60.173145]}
Kannelmäen liikuntapuisto / Ulkokuntosali
{'type': 'Point', 'coordinates': [24.878023, 60.24363]}
Vuosaaren liikuntapuisto / Ulkokuntosali
{'t

In [6]:
# The "plan":
#   - Create a "database" of units that can be used for recommendation
#   - Units should have attributes
#       -> Some come from parents (e.g., outdoor/aerobic/etc)
#       -> Some should be added from unit information (e.g., season, extreme, opening hours)
# How to do this?
#   1) Download the units together with the hierarchy, like in servicemap list
#   2) Give the units attributes based on their parents in the hierarchy
#   3) Give the units attributes based on their individual classes

# Go through the service node hierarchy of "sports" class, to download
# similar representation as in servicemap UI

def get_node(url):
    resp = requests.get(url)    
    resp.raise_for_status()
    resp = resp.json()
    return resp['results'][0]

def traverse_children(node, attributes, path):
    node_name = node['name']['en']
    if node_name in ignored:
       return
    
    if node_name in class_related_attributes.keys():
        new_attributes = attributes.union(class_related_attributes[node['name']['en']])
    else:
       new_attributes = attributes
    new_path = path + [node_name]

    if node['children']:
      for child_id in node['children']:
        url = f'https://api.hel.fi/servicemap/v2/service_node/?id={child_id}'
        traverse_children(get_node(url), new_attributes, new_path)
    else:
        # Leafs have related services
        for service in node['related_services']:
            url = f'https://api.hel.fi/servicemap/v2/unit/?service={service}&only=id,name'
            resp = requests.get(url)    
            resp.raise_for_status()
            resp = resp.json()
            resp = resp['results']

            for unit in resp:
                unit['attributes'] = [att.name for att in new_attributes]
                unit['path'] = new_path
                units.append(unit)

# Classes to ignore (in current view useless sports)
ignored = {'Animal sports areas', 
           'Electronic services and forms related to sports', 
           'Physical education guidance',
           'Boating, aviation and motor sports',
           'Recreational destinations and services',
           'Senior sports',
           
           'Shooting sports facilities',
           'Bowling alleys',
           'Ski jumping hills',
           'Curling sheet'
           }

# TODO: Replace Traumatic with safe for pregnant
ActivityAttributes = Enum(
   'ActivityAttributes',
   ['AEROBIC', 'STRENGTH', 'OUTDOOR', 'WINTER', 'SUMMER', 'TRAUMATIC']
)

class_related_attributes = {
   #
   'Cross-country sports facilities': {ActivityAttributes.OUTDOOR},
   ## Sports and outdoor recreations routes
   'Climbing venues' : {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH, ActivityAttributes.TRAUMATIC},
   'Ski slopes and downhill ski resorts': {ActivityAttributes.AEROBIC, ActivityAttributes.WINTER, ActivityAttributes.TRAUMATIC},
   'Sports and outdoor recreations routes': set(),
   ###
   'Jogging track' : {ActivityAttributes.AEROBIC},
   'Outdoor route': {ActivityAttributes.AEROBIC},
   'Ski track': {ActivityAttributes.AEROBIC, ActivityAttributes.WINTER, ActivityAttributes.TRAUMATIC},
   'Nature trail': {ActivityAttributes.AEROBIC},
   'Cross-country biking route': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Canoe route': {ActivityAttributes.AEROBIC, ActivityAttributes.SUMMER},
   'Biking route': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Hiking route': {ActivityAttributes.AEROBIC},
   'Water route': {ActivityAttributes.AEROBIC},
   ##
   'Cross-country ski resorts': {ActivityAttributes.AEROBIC, ActivityAttributes.WINTER},
   'Orienteering area': {ActivityAttributes.AEROBIC},
   ###
   'Ski orienteering area' : {ActivityAttributes.WINTER, ActivityAttributes.TRAUMATIC},
   'Mountain bike orienteering area': {ActivityAttributes.SUMMER, ActivityAttributes.TRAUMATIC},
   'Orienteering area': set(),
   #
   'Indoor sports facilites': set(),
   ##
   'Ice-skating arenas': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Fitness centres and sports halls': set(),
   ###
   'Martial arts halls': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Fitness centre': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Gym': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Gymnastics hall': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Weight training hall': {ActivityAttributes.STRENGTH},
   ##
   'Sports halls': {ActivityAttributes.AEROBIC},
   ###
   'Football hall': {ActivityAttributes.TRAUMATIC},
   'Skateboarding hall': {ActivityAttributes.TRAUMATIC},
   'Squash hall': {ActivityAttributes.TRAUMATIC},
   'Badminton hall': {ActivityAttributes.TRAUMATIC},
   'Tennis hall': {ActivityAttributes.TRAUMATIC},
   ##
   'Indoor venues of various sports' : set(),
   ###
   'Fencing venue': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Parkour hall': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Table tennis venue': {ActivityAttributes.AEROBIC},
   'Indoor shooting range': set(),
   'Indoor climbing wall': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH, ActivityAttributes.TRAUMATIC},
   'Dance studio': {ActivityAttributes.AEROBIC},
   'Artistic gymnastics facility': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Stand-alone athletics venue': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   #
   'Outdoor fields and sports parks' : {ActivityAttributes.OUTDOOR},
   ##
   'Golf courses': set(),
   'Ice sports areas and sites with natural ice': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC, ActivityAttributes.WINTER},
   'Neighbourhood sports facilities and parks': set(),
   ###
   'Disc golf course': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Sports park': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Neighbourhood sports area': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   'Parkour area': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH, ActivityAttributes.TRAUMATIC},
   'Cycling area': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Velodrome': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Skateboarding/roller-blading rink': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Fitness training park': {ActivityAttributes.AEROBIC, ActivityAttributes.STRENGTH},
   ##
   'Ball games courts': {ActivityAttributes.AEROBIC, ActivityAttributes.TRAUMATIC},
   'Athletics fields and venues': {ActivityAttributes.AEROBIC},
   #
   'Water sports facilities': {ActivityAttributes.AEROBIC},
   ##
   'Open air pools and beaches': {ActivityAttributes.OUTDOOR},
   ###
   'Winter swimming area': {ActivityAttributes.WINTER},
   ##
   'Indoor swimming pools, halls and spas': set()
}

units = []

# Sports class id 
root_id = 551
url = f'https://api.hel.fi/servicemap/v2/service_node/?id={root_id}'
root_node = get_node(url)

traverse_children(root_node, set(), [])

print(units)

[{'id': 53167, 'name': {'fi': 'Tussari'}, 'attributes': ['AEROBIC', 'STRENGTH', 'OUTDOOR', 'TRAUMATIC'], 'path': ['Sports and physical exercise', 'Cross-country sports facilities', 'Climbing venues', 'Climbing rock']}, {'id': 46001, 'name': {'fi': 'Rollarit'}, 'attributes': ['AEROBIC', 'STRENGTH', 'OUTDOOR', 'TRAUMATIC'], 'path': ['Sports and physical exercise', 'Cross-country sports facilities', 'Climbing venues', 'Climbing rock']}, {'id': 46000, 'name': {'fi': 'Käärmekallio'}, 'attributes': ['AEROBIC', 'STRENGTH', 'OUTDOOR', 'TRAUMATIC'], 'path': ['Sports and physical exercise', 'Cross-country sports facilities', 'Climbing venues', 'Climbing rock']}, {'id': 54344, 'name': {'fi': 'Postipuun koulu / Ulkokiipeilyseinä', 'sv': 'Postipuun koulu / Utomhusklättervägg', 'en': 'Postipuun koulu / Outdoor climbing wall'}, 'attributes': ['AEROBIC', 'STRENGTH', 'OUTDOOR', 'TRAUMATIC'], 'path': ['Sports and physical exercise', 'Cross-country sports facilities', 'Climbing venues', 'Open-air climbin

In [8]:
# Now create an index for searching purposes
index = {a.name: set() for a in ActivityAttributes}
for unit in units:
    for attribute in unit['attributes']:
        index[attribute].add(unit['id'])


In [11]:
activity_json = json.dumps({'data': units, 'index': index}, default=list)
print(activity_json)


{"data": [{"id": 53167, "name": {"fi": "Tussari"}, "attributes": ["AEROBIC", "STRENGTH", "OUTDOOR", "TRAUMATIC"], "path": ["Sports and physical exercise", "Cross-country sports facilities", "Climbing venues", "Climbing rock"]}, {"id": 46001, "name": {"fi": "Rollarit"}, "attributes": ["AEROBIC", "STRENGTH", "OUTDOOR", "TRAUMATIC"], "path": ["Sports and physical exercise", "Cross-country sports facilities", "Climbing venues", "Climbing rock"]}, {"id": 46000, "name": {"fi": "K\u00e4\u00e4rmekallio"}, "attributes": ["AEROBIC", "STRENGTH", "OUTDOOR", "TRAUMATIC"], "path": ["Sports and physical exercise", "Cross-country sports facilities", "Climbing venues", "Climbing rock"]}, {"id": 54344, "name": {"fi": "Postipuun koulu / Ulkokiipeilysein\u00e4", "sv": "Postipuun koulu / Utomhuskl\u00e4tterv\u00e4gg", "en": "Postipuun koulu / Outdoor climbing wall"}, "attributes": ["AEROBIC", "STRENGTH", "OUTDOOR", "TRAUMATIC"], "path": ["Sports and physical exercise", "Cross-country sports facilities", "C

In [12]:
index

{'AEROBIC': {39238,
  39250,
  39255,
  39270,
  39290,
  39300,
  39396,
  39432,
  39465,
  39493,
  39499,
  39530,
  39535,
  39544,
  39552,
  39556,
  39560,
  39564,
  39585,
  39596,
  39608,
  39617,
  39625,
  39690,
  39760,
  39767,
  39773,
  39785,
  39790,
  39804,
  39816,
  39825,
  39829,
  39853,
  39945,
  39947,
  39978,
  39980,
  40000,
  40001,
  40025,
  40118,
  40128,
  40134,
  40142,
  40150,
  40157,
  40164,
  40182,
  40188,
  40195,
  40196,
  40197,
  40204,
  40227,
  40233,
  40237,
  40256,
  40258,
  40338,
  40355,
  40360,
  40386,
  40432,
  40454,
  40498,
  40507,
  40519,
  40524,
  40526,
  40550,
  40557,
  40559,
  40565,
  40569,
  40573,
  40592,
  40601,
  40616,
  40617,
  40623,
  40624,
  40686,
  40725,
  40731,
  40774,
  40802,
  40808,
  40823,
  40832,
  40838,
  40856,
  40882,
  40909,
  40931,
  40950,
  40961,
  40970,
  40975,
  40982,
  40996,
  40997,
  41008,
  41028,
  41047,
  41076,
  41097,
  41102,
  41129,
  41168,